In [3]:
import pandas as pd
import numpy as np

# Deck

## Requirement

- suits not required
- multiple decks
- need to be able to shuffle the deck
- need to draw a card from the deck

## Plan

- class: deck
    - can take number of decks as a variable
    - default 1 deck
- method: shuffle
- method: deal
- attribute: list of all the cards in the 'deck'

## Code

In [4]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def shuffle(self):
        np.random.shuffle(self.deck)
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [5]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']


# Calculate points

## Requirement

- Take a hand
- Calculate how many points there are in the hand
    - J, Q, K are 10
    - Ace could be 1 or 11
        - How do we decide?
        - If 11 doesn't bust, then 11, else 1
            - We have to calculate the rest of the cards first

## Plan

- Remove Aces in the hand
    - Track how many Aces there were
- Count J, Q, K as 10

## Code

In [6]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [7]:
calculate_points([5, 5, "A"])

21

# Simulate game

## Requirements

- play one game
- return the data we need

In [8]:
data_dictionary = pd.read_csv("data_dictionary.csv")
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_3_card,Int,Dealer's 3 card points (unless dealer has less...
6,dealer_final,Int,The dealer's final points
7,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
8,player_card_one,Int,The player's first card
9,player_card_two,Int,The player's second card


## Outcomes

- player gets blackjack, dealer didn't
    - player wins
- dealer gets blackjack, player didn't
    - player loses
- player hits and busts
    - player loses
- player hits, dealer hits and busts
    - player wins
- player hits, dealer hits/ dealer >= hard 17
    - player > dealer
        - player wins
    - player < dealer
        - player loses
    - player == dealer
        - draw

## Plan

- create deck
- deal initial hand
- check if anyone has a blackjack
- no blackjacks, player goes first
    - in any case, if player has anything <= 11, hit
    - if anything 12 or higher
        - random
            - randomize whether to hit or not if smaller than 18
        - recommended
            - if dealer showing <= 6 and player <= 16, hit
- once player is done, dealer goes
    - if dealer has anything smaller than 17, he must hit
    - if dealer has a soft 17, he must hit

In [9]:
def sim_game(num_decks=1, strategy=0):
    game_deck = Deck(num_decks=num_decks)
    game_deck.shuffle()

    dealer_hand = []
    player_hand = []

    # deal 2 cards each
    for i in range(2):
        game_deck.deal(player_hand)
        game_deck.deal(dealer_hand)

    # get their initial points
    # copy so that our hand is not changed
    player_initial = calculate_points(player_hand.copy())
    dealer_initial = calculate_points(dealer_hand.copy())

    # get dealer open card
    dealer_open = dealer_hand[0]

    # get cards from player
    player_card_one = player_hand[0]
    player_card_two = player_hand[1]

    # Change them to numerics
    if dealer_open in ["J", "Q", "K"]:
        dealer_open = 10
    elif dealer_open == "A":
        dealer_open = 1

    if player_card_one in ["J", "Q", "K"]:
        player_card_one = 10
    elif player_card_one == "A":
        player_card_one = 1

    if player_card_two in ["J", "Q", "K"]:
        player_card_two = 10
    elif player_card_two == "A":
        player_card_two = 1

    dealer_hit = 0
    dealer_num_hits = 0
    player_hit = 0
    player_num_hits = 0
    player_busts = 0
    dealer_busts = 0
    dealer_final = calculate_points(dealer_hand.copy())
    player_final = calculate_points(player_hand.copy())


    # if anyone got a blackjack, the game should end
    if player_initial != 21 and dealer_initial != 21:
        # if neither of them got a blackjack game continues
        # the player goes first

        # If player <= 11, hit
        while calculate_points(player_hand.copy()) <= 11:
            player_hit = 1
            player_num_hits += 1
            game_deck.deal(player_hand)

        # If strategy is random, randomize hit for 18 and below
        if strategy == 0:
            while calculate_points(player_hand.copy()) <= 18:
                if np.random.random() < 0.5:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)
        # If strategy is recommended, stand on 17 and above
        else:
            if dealer_open <= 6:
                while calculate_points(player_hand.copy()) <= 16:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)

        # update player's final and busts
        player_final = calculate_points(player_hand.copy())
        player_busts = player_final > 21
        # dealer's turn

        # If player didn't bust
        if not player_busts:
            # If dealer < 17, hit
            while calculate_points(dealer_hand.copy()) < 17:
                dealer_hit = 1
                dealer_num_hits += 1
                game_deck.deal(dealer_hand)

            # If dealer has a soft 17, hit
            if calculate_points(dealer_hand.copy()) == 17:
                if "A" in dealer_hand:
                    dealer_copy = dealer_hand.copy()
                    dealer_copy.remove("A")
                    if calculate_points(dealer_copy.copy()) == 6:
                        dealer_hit = 1
                        dealer_num_hits += 1
                        game_deck.deal(dealer_hand)

        # update dealer's final and busts
        dealer_final = calculate_points(dealer_hand.copy())
        dealer_busts = dealer_final > 21

    player_loses = 0
    draw = 0
    player_wins = 0

    # Get three card totals
    if len(player_hand) >= 3:
        player_3_card = calculate_points(player_hand.copy()[:3])
    else:
        player_3_card = calculate_points(player_hand.copy())

    if len(dealer_hand) >= 3:
        dealer_3_card = calculate_points(dealer_hand.copy()[:3])
    else:
        dealer_3_card = calculate_points(dealer_hand.copy())

    # Check who wins
    if player_initial == 21 and dealer_initial != 21:
        player_wins = 1
    elif dealer_initial == 21 and player_initial != 21:
        player_loses = 1
    elif player_busts:
        player_loses = 1
    elif dealer_busts:
        player_wins = 1
    elif player_final > dealer_final:
        player_wins = 1
    elif player_final < dealer_final:
        player_loses = 1
    elif player_final == dealer_final:
        draw = 1

    return np.array([num_decks, dealer_open, dealer_initial, dealer_hit, dealer_num_hits, dealer_3_card, dealer_final, int(dealer_busts), player_card_one, player_card_two, player_initial, player_hit, player_num_hits, player_3_card, player_final, int(player_busts), player_loses, draw, player_wins, strategy, str(dealer_hand), str(player_hand)])

In [10]:
print (sim_game())

['1' '1' '20' '0' '0' '20' '20' '0' '10' '4' '14' '1' '3' '15' '20' '0' '0'
 '1' '0' '0' "['A', 9]" "['J', 4, 'A', 3, 2]"]


# Generate a dataframe

## Requirements

- run sim_game() for some number of times and save the output to a df

In [184]:
data_dictionary.Feature.values

array(['num_decks', 'dealer_open', 'dealer_initial', 'dealer_hit',
       'dealer_num_hits', 'dealer_final', 'dealer_busts', 'player_inital',
       'player_hit', 'player_num_hits', 'player_final', 'player_busts',
       'player_loses', 'draw', 'player_wins', 'strategy', 'dealer_hand',
       'player_hand'], dtype=object)

In [212]:
def gen_data(num_decks=1, df_size=5000, strategy=0):
    return np.array([sim_game(num_decks=num_decks, strategy=strategy) for _ in range(df_size)])

In [213]:
def gen_df(data):
    tmp = pd.DataFrame(data, columns=data_dictionary.Feature.values)
    tmp[tmp.columns.values[:-2]] = tmp[tmp.columns.values[:-2]].astype(int)
    return tmp

In [216]:
df = gen_df(gen_data(num_decks=4))

In [224]:
sub = gen_df(gen_data(num_decks=4, strategy=1))

In [225]:
df = pd.concat([df,sub])

In [180]:
# Combine old files
old_df = pd.read_csv("blackjack.csv")
df = pd.concat([old_df, df])

In [226]:
# save to file
df.to_csv("blackjack.csv", index=False)

# Testing our code

In [229]:
df = pd.read_csv("blackjack.csv")

In [230]:
df.shape

(20000, 18)

In [231]:
# When player_loses = 1, draw = 0, player_wins = 0
print (df.draw[df.player_loses == 1].value_counts())
print (df.player_wins[df.player_loses == 1].value_counts())

0    10846
Name: draw, dtype: int64
0    10846
Name: player_wins, dtype: int64


In [232]:
# When player_loses = 0, draw = 1, player_wins = 0
print (df.player_loses[df.draw == 1].value_counts())
print (df.player_wins[df.draw == 1].value_counts())

0    1479
Name: player_loses, dtype: int64
0    1479
Name: player_wins, dtype: int64


In [233]:
# When player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.player_wins == 1].value_counts())
print (df.draw[df.player_wins == 1].value_counts())

0    7675
Name: player_loses, dtype: int64
0    7675
Name: draw, dtype: int64


In [234]:
# When dealer_busts = 1, player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.dealer_busts == 1].value_counts())
print (df.draw[df.dealer_busts == 1].value_counts())
print (df.player_wins[df.dealer_busts == 1].value_counts())

0    3565
Name: player_loses, dtype: int64
0    3565
Name: draw, dtype: int64
1    3565
Name: player_wins, dtype: int64


In [235]:
# When player_busts = 1, player_loses = 1, draw = 0, player_wins = 0, dealer_hit = 0, dealer_busts = 0
print (df.player_loses[df.player_busts == 1].value_counts())
print (df.draw[df.player_busts == 1].value_counts())
print (df.player_wins[df.player_busts == 1].value_counts())
print (df.dealer_hit[df.player_busts == 1].value_counts())
print (df.dealer_busts[df.player_busts == 1].value_counts())

1    5904
Name: player_loses, dtype: int64
0    5904
Name: draw, dtype: int64
0    5904
Name: player_wins, dtype: int64
0    5904
Name: dealer_hit, dtype: int64
0    5904
Name: dealer_busts, dtype: int64


In [236]:
# When dealer_busts = 0 and player_busts = 0, if player_final > dealer final, player_loses = 0, draw = 0, player_wins = 1
sub = df[(df.player_busts == 0) & (df.dealer_busts == 0)]
print (sub.player_loses[sub.player_final > sub.dealer_final].value_counts())
print (sub.draw[sub.player_final > sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final > sub.dealer_final].value_counts())

0    4110
Name: player_loses, dtype: int64
0    4110
Name: draw, dtype: int64
1    4110
Name: player_wins, dtype: int64


In [237]:
# When dealer_busts = 0 and player_busts = 0, if player_final = dealer final, player_loses = 0, draw = 1, player_wins = 0
print (sub.player_loses[sub.player_final == sub.dealer_final].value_counts())
print (sub.draw[sub.player_final == sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final == sub.dealer_final].value_counts())

0    1479
Name: player_loses, dtype: int64
1    1479
Name: draw, dtype: int64
0    1479
Name: player_wins, dtype: int64


In [238]:
# When dealer_busts = 0 and player_busts = 0, if player_final < dealer final, player_loses = 1, draw = 0, player_wins = 0
print (sub.player_loses[sub.player_final < sub.dealer_final].value_counts())
print (sub.draw[sub.player_final < sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final < sub.dealer_final].value_counts())

1    4942
Name: player_loses, dtype: int64
0    4942
Name: draw, dtype: int64
0    4942
Name: player_wins, dtype: int64


In [239]:
# When player_final <= 21, player_busts == 0
print (df.player_busts[df.player_final <= 21].value_counts())

0    14096
Name: player_busts, dtype: int64


In [240]:
# When dealer_final <= 21, dealer_busts == 0
print (df.dealer_busts[df.dealer_final <= 21].value_counts())

0    16435
Name: dealer_busts, dtype: int64


In [241]:
# When player_hit == 1, player_num_hits >= 1
print (df.player_num_hits[df.player_hit == 1].value_counts())

1    7257
2    3299
3     839
4     115
5      13
6       1
Name: player_num_hits, dtype: int64


In [242]:
# When dealer_hit == 1, dealer_num_hits >= 1
print (df.dealer_num_hits[df.dealer_hit == 1].value_counts())

1    5674
2    2096
3     389
4      39
6       1
5       1
Name: dealer_num_hits, dtype: int64


In [243]:
df.dealer_hit.value_counts()

0    11800
1     8200
Name: dealer_hit, dtype: int64

In [244]:
df.dealer_num_hits.value_counts()

0    11800
1     5674
2     2096
3      389
4       39
6        1
5        1
Name: dealer_num_hits, dtype: int64

In [245]:
df.player_hit.value_counts()

1    11524
0     8476
Name: player_hit, dtype: int64

In [246]:
df.player_num_hits.value_counts()

0    8476
1    7257
2    3299
3     839
4     115
5      13
6       1
Name: player_num_hits, dtype: int64